In [7]:
import sqlite3
import csv
import os

try:
    # 파일 존재 여부 확인
    if not os.path.exists('recipes_3.csv'):
        print("Error: recipes_3.csv 파일을 찾을 수 없습니다.")
        exit(1)
    
    # 데이터베이스 연결
    conn = sqlite3.connect('recipes.db')
    cursor = conn.cursor()

    # 테이블 생성 (id를 AUTOINCREMENT로 설정)
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS recipes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            image TEXT,
            ingredients TEXT,
            steps TEXT,
            source TEXT
        )
    ''')

    # CSV 데이터 읽기 및 삽입
    with open('recipes_3.csv', 'r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            # id 컬럼 제외하고 삽입
            cursor.execute('''
                INSERT INTO recipes (name, image, ingredients, steps, source)
                VALUES (?, ?, ?, ?, ?)
            ''', (row['name'], row['image'], row['ingredients'], 
                  row['steps'], row['source']))

    conn.commit()
    print("데이터베이스에 성공적으로 데이터를 삽입했습니다.")
    
except Exception as e:
    print(f"예상치 못한 오류: {e}")
finally:
    if 'conn' in locals():
        conn.close()


데이터베이스에 성공적으로 데이터를 삽입했습니다.


In [ ]:
import pandas as pd
import sqlite3
import json

def recipes_csv_to_sqlite(recipes_3.csv, recipes.db):
    """
    레시피 CSV를 React Native용 SQLite로 변환 (최적화)
    """
    # CSV 읽기
    df = pd.read_csv(recipes_3.csv)
    
    # SQLite 연결
    conn = sqlite3.connect(recipes.db)
    cursor = conn.cursor()
    
    # 레시피 테이블 생성
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS recipes (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            image TEXT,
            ingredients TEXT,
            steps TEXT,
            source TEXT
        )
    ''')
    
    # 데이터 삽입
    for _, row in df.iterrows():
        cursor.execute('''
            INSERT INTO recipes (id, name, image, ingredients, steps, source)
            VALUES (?, ?, ?, ?, ?, ?)
        ''', (
            row['id'],
            row['name'],
            row['image'],
            row['ingredients'],
            row['steps'],
            row['source']
        ))
    
    # 검색을 위한 인덱스 생성
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_recipe_name ON recipes(name)')
    
    conn.commit()
    conn.close()
    
    print(f"레시피 데이터베이스가 {recipes.db}에 생성되었습니다.")

# 사용 예시
recipes_csv_to_sqlite('recipes_3.csv', 'recipes.db')
